In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/buliding-density/validation_data_with_density.csv
/kaggle/input/buliding-density/training_data_with_density.csv
/kaggle/input/satellite-data/validation_data_with_satellite.csv
/kaggle/input/satellite-data/training_data_with_satellite.csv


In [2]:
# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# Combine Datasets

In [3]:
train_density = pd.read_csv("/kaggle/input/buliding-density/training_data_with_density.csv")
train_density.head()

,Longitude,Latitude,datetime,UHI Index,density
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8


In [4]:
train_satellite = pd.read_csv("/kaggle/input/satellite-data/training_data_with_satellite.csv")
train_satellite.head()

,Longitude,Latitude,datetime,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,825.0,990.0,1144.0,1172.0,1435.0,1688.0,1688.0,1830.0,1688.0,1788.0,1540.0,0.219187,-0.011609,-0.230666,38.393941
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,825.0,990.0,1144.0,1172.0,1435.0,1688.0,1688.0,1830.0,1688.0,1788.0,1540.0,0.219187,-0.011609,-0.230666,38.393941
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,825.0,622.0,785.0,744.0,1114.0,2006.0,2138.0,2318.0,2353.0,1566.0,1170.0,0.514043,-0.193615,-0.494038,37.785534
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,825.0,619.0,750.0,738.0,1114.0,2006.0,2138.0,2302.0,2353.0,1566.0,1170.0,0.514474,-0.190279,-0.508519,37.785534
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,825.0,556.0,758.0,660.0,1056.0,1891.0,2141.0,2280.0,2259.0,1658.0,1240.0,0.551020,-0.157948,-0.500987,37.358281


In [5]:
train_satellite = train_satellite.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat = pd.concat([train_density, train_satellite], axis = 1)
train_concat.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,1688.0,1688.0,1830.0,1688.0,1788.0,1540.0,0.219187,-0.011609,-0.230666,38.393941
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,1688.0,1688.0,1830.0,1688.0,1788.0,1540.0,0.219187,-0.011609,-0.230666,38.393941
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,825.0,622.0,785.0,744.0,1114.0,2006.0,2138.0,2318.0,2353.0,1566.0,1170.0,0.514043,-0.193615,-0.494038,37.785534
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,825.0,619.0,750.0,738.0,1114.0,2006.0,2138.0,2302.0,2353.0,1566.0,1170.0,0.514474,-0.190279,-0.508519,37.785534
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,825.0,556.0,758.0,660.0,1056.0,1891.0,2141.0,2280.0,2259.0,1658.0,1240.0,0.551020,-0.157948,-0.500987,37.358281


In [6]:
validation_density = pd.read_csv("/kaggle/input/buliding-density/validation_data_with_density.csv")
validation_density.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,density
0,-73.971665,40.788763,NaN,12
1,-73.971928,40.788875,NaN,12
2,-73.967080,40.789080,NaN,4
3,-73.972550,40.789082,NaN,12
4,-73.969697,40.787953,NaN,9


In [7]:
validation_satellite = pd.read_csv("/kaggle/input/satellite-data/validation_data_with_satellite.csv")
validation_satellite.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.971665,40.788763,NaN,794.0,511.0,568.0,527.0,1112.0,2323.0,2649.0,2360.0,2784.0,1741.0,1218.0,0.634915,-0.150939,-0.612022,36.202990
1,-73.971928,40.788875,NaN,1221.0,494.0,661.0,497.0,1315.0,2652.0,2964.0,3662.0,3102.0,2124.0,1541.0,0.761000,-0.265814,-0.694194,36.202990
2,-73.967080,40.789080,NaN,1049.0,841.0,968.0,1056.0,1028.0,1418.0,1610.0,1402.0,1636.0,1634.0,1198.0,0.140765,0.076416,-0.183122,36.069687
3,-73.972550,40.789082,NaN,1189.0,903.0,984.0,1108.0,1971.0,1978.0,1970.0,1478.0,2049.0,2303.0,2219.0,0.143078,0.218196,-0.200650,36.886594
4,-73.969697,40.787953,NaN,1018.0,716.0,1046.0,917.0,1286.0,2382.0,2778.0,3038.0,2912.0,2102.0,1705.0,0.536283,-0.182101,-0.487757,34.500816


In [8]:
validation_satellite = validation_satellite.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat = pd.concat([validation_density, validation_satellite], axis=1)
validation_concat.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.971665,40.788763,NaN,12,794.0,511.0,568.0,527.0,1112.0,2323.0,2649.0,2360.0,2784.0,1741.0,1218.0,0.634915,-0.150939,-0.612022,36.202990
1,-73.971928,40.788875,NaN,12,1221.0,494.0,661.0,497.0,1315.0,2652.0,2964.0,3662.0,3102.0,2124.0,1541.0,0.761000,-0.265814,-0.694194,36.202990
2,-73.967080,40.789080,NaN,4,1049.0,841.0,968.0,1056.0,1028.0,1418.0,1610.0,1402.0,1636.0,1634.0,1198.0,0.140765,0.076416,-0.183122,36.069687
3,-73.972550,40.789082,NaN,12,1189.0,903.0,984.0,1108.0,1971.0,1978.0,1970.0,1478.0,2049.0,2303.0,2219.0,0.143078,0.218196,-0.200650,36.886594
4,-73.969697,40.787953,NaN,9,1018.0,716.0,1046.0,917.0,1286.0,2382.0,2778.0,3038.0,2912.0,2102.0,1705.0,0.536283,-0.182101,-0.487757,34.500816


# Select features 

In [9]:
features = ['B01','B06','NDVI','NDBI','NDWI','LST','density']
train_df = train_concat[features + ["UHI Index"]]
train_df

,B01,B06,NDVI,NDBI,NDWI,LST,density,UHI Index
0,825.0,1688.0,0.219187,-0.011609,-0.230666,38.393941,8,1.030289
1,825.0,1688.0,0.219187,-0.011609,-0.230666,38.393941,8,1.030289
2,825.0,2006.0,0.514043,-0.193615,-0.494038,37.785534,8,1.023798
3,825.0,2006.0,0.514474,-0.190279,-0.508519,37.785534,8,1.023798
4,825.0,1891.0,0.551020,-0.157948,-0.500987,37.358281,8,1.021634
...,...,...,...,...,...,...,...,...
11224,432.0,2545.0,0.727473,-0.241216,-0.628594,30.293234,0,0.972470
11225,432.0,2545.0,0.766208,-0.303371,-0.659820,30.293234,0,0.972470
11226,432.0,2545.0,0.766208,-0.303371,-0.659820,30.440209,0,0.981124
11227,432.0,2545.0,0.766208,-0.303371,-0.659820,30.440209,0,0.981245


# Remove duplicates from training data

In [10]:
# Remove duplicate rows from the DataFrame based on specified columns and keep the first occurrence
for col in features:
    # Check if the value is a numpy array and has more than one dimension
    train_df[col] = train_df[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)

# Now remove duplicates
uhi_data = train_df.drop_duplicates(subset=features, keep='first')
uhi_data.head()

<ipython-input-10-cc52a2d8b8e0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[col] = train_df[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)


,B01,B06,NDVI,NDBI,NDWI,LST,density,UHI Index
0,825.0,1688.0,0.219187,-0.011609,-0.230666,38.393941,8,1.030289
2,825.0,2006.0,0.514043,-0.193615,-0.494038,37.785534,8,1.023798
3,825.0,2006.0,0.514474,-0.190279,-0.508519,37.785534,8,1.023798
4,825.0,1891.0,0.551020,-0.157948,-0.500987,37.358281,8,1.021634
6,825.0,1891.0,0.612442,-0.219765,-0.553026,37.358281,8,1.015143


In [11]:
uhi_data.shape

(9054, 8)

In [12]:
# Resetting the index of the dataset
uhi_data=uhi_data.reset_index(drop=True)

In [13]:
uhi_data.isna().sum()

B01          0
B06          0
NDVI         0
NDBI         0
NDWI         0
LST          0
density      0
UHI Index    0
dtype: int64

# Model Building

In [14]:
# Split the data into features (X) and target (y), and then into training and testing sets
X = uhi_data.drop(columns=['UHI Index']).values
y = uhi_data ['UHI Index'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=123)

**Feature Scaling**

In [15]:
# Scale the training and test data using standardscaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Model Training**

In [16]:
# Define the model
catboost = CatBoostRegressor(loss_function='RMSE', random_state=42, verbose=0)

# Define the hyperparameter grid for RandomizedSearchCV
param_grid = {
    'n_estimators': [200, 300, 500, 700, 900],          # Number of trees
    'depth': [5, 10, 30, 50, 100],                     # Maximum depth of trees
    'learning_rate': [0.01, 0.05, 0.1, 0.2],           # Step size shrinkage
    'subsample': [0.6, 0.8, 1.0],                      # Fraction of samples used per tree
    'colsample_bylevel': [0.6, 0.8, 1.0],              # Fraction of features used per level
    'min_data_in_leaf': [1, 2, 5, 10],                 # Minimum sum of instance weight needed in a leaf
    'l2_leaf_reg': [1, 3, 5, 10]                       # L2 regularization term
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=catboost,
                                   param_distributions=param_grid,
                                   n_iter=1000,           # Number of random combinations to test
                                   cv=5,                 # 5-fold cross-validation
                                   n_jobs=-1,            # Use all available cores
                                   verbose=2,            # Print progress
                                   random_state=42,      # For reproducibility
                                   scoring='r2')         # Use R-squared for scoring

# Fit the model
random_search.fit(X_train, y_train)

# Print the best hyperparameters found by RandomizedSearchCV
print("Best Hyperparameters found: ", random_search.best_params_)

# Retrieve the best model
best_catboost = random_search.best_estimator_

# Print best model details
print(f"Best model has {best_catboost.get_params()['n_estimators']} trees and max depth of {best_catboost.get_params()['depth']}.")


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


KeyboardInterrupt: 

**Model Evaluation**

In [ ]:
y_pred = best_catboost.predict(X_test)
Y_test = y_test.tolist()
print("R² Score:", r2_score(Y_test, y_pred))

# Submission

In [ ]:
validation_concat.head()

In [ ]:
submission_val_data = validation_concat[features]
submission_val_data.head()

In [ ]:
# Feature Scaling 
submission_val_data = submission_val_data.values
transformed_submission_data = sc.transform(submission_val_data)

In [ ]:
#Making predictions
final_predictions = best_catboost.predict(transformed_submission_data)
final_prediction_series = pd.Series(final_predictions)

In [ ]:
#Combining the results into dataframe
submission_df = pd.DataFrame({'Longitude':validation_concat['Longitude'].values, 'Latitude':validation_concat['Latitude'].values, 'UHI Index':final_prediction_series.values})

In [ ]:
#Displaying the sample submission dataframe
submission_df.head()

In [ ]:
#Dumping the predictions into a csv file.
submission_df.to_csv("submission.csv",index = False)